In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from random import randint

In [2]:
#Put in pandas df for analysis
columns = ["Slug","WG","WI","MG","G","AST","GK","RK","SH","SH%","PEN","PEN%","xG","DA","P","P%","KP","CRS","CRS%","DRB","OFF","BVR","BVT","D","D%","DL","DL%","TCK","TCK%","INT","BVL","BVE","FOUT"]
df = pd.DataFrame(columns=columns)
df

,Slug,WG,WI,MG,G,AST,GK,RK,SH,SH%,...,D,D%,DL,DL%,TCK,TCK%,INT,BVL,BVE,FOUT


In [4]:
#retrieve available players
slugs = []


for i in range(18):
    page = i+1
    interval = randint(1,10)
    print("Waiting for",interval," seconds.")
    time.sleep(interval)
    headers = {
        'authority': 'www.footballtransfers.com',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9,nl-NL;q=0.8,nl;q=0.7',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        # 'cookie': 'FootballTransfers_Language=2; _sp_v1_uid=1:542:959c4b66-2f54-4355-9032-772b8f66b930; _sp_v1_ss=1:H4sIAAAAAAAAAItWqo5RKimOUbKKxs_IAzEMamN1YpRSQcy80pwcILsErKC6lpoSSrEA-EAOLpYAAAA%3D; euconsent-v2=CPtJWIAPtJWIAAGABCENDECsAP_AAH_AABpYIuNN_H9ebXtjef596Pt0eY0e91j2ruQyDheNk6UByJ7W7KwX0WA7NA3qhqAImR4EukIBAQNEGEHUDQmgYIEVKTGMAkyEpyIKI6IEkmERAmUYGFtPG0EB-YCQz1ZdFnZh0C4I_9r-x8y3_8df3_NJ98_-wAAQAAAAAAAAQAAAAAAAAIAAgAAAAAAAAAAAQQTQLMNS4moLF89CSUMAoURowriBgAEBgFAMJZEIBETgp2VgEgoB0AAMUAUgRMhxBYAwCAACAAJCoCBJCwQKCQCEQAAgARAAAA0QBJAIiAEAgAJAeFgACAAEghgyIAMjDggAISDSlgAAEsKqjHyBHooASAAAgAAAAAAAAAAAAAAAAAAABATCQAgDFAHyCgQQAAgAKgBOAFAAKgAWAA7AB6AEIAI4AUgArgBtAEUAJWAVoBXADCAGcANEAaoA4wB1gEIAJKASeAnQCdgFNAKhAV4BXwC7AGcANsAcAA44BygDmgHiAPUAe0A_cCOgI7ASWAloBLwCYgFCAKXAUyAr4BYgCyQFogLsAXoAwgBi4DGQGOAMeAZ-A0wBpwDVwGtANeAbWGgAgHyDgMAAAgAUgBOAFAAKgAWQA8AD0AIQARAAjgBMACuAG0AOcAdwB4gEQARIApQBWgDOAHGAOcAdYBJwCdQFeAV8AuwBnADbAHAAOOAcoA5oB4gD1AH7AS8AoQBaIC5QF0ALsAXoAxkBjwDJAGmANOAa0A3IRABAPkJAZgABAApACcAKAAVAAsgB4AHwAQgAiABHACYAFcANoAcwA7wCIAIkAUoArQBnADjAHOAOsAk8BOgE7AKhAV4BXwC7AGcANsAcAA44BygDmgHiAPUAfsBHYCXgFCALEAWiAuUBdAC7AF6AMZAY8AyQBpgDTgGtANrFgFQAVAAsACEAG0APAArQBnADjAJPAV4BXwC7AGcAOaAeIA_YCXgFogLoAXoAx4BrQ0AqACoAFgAQgAzABtgFaAVwAzgBxgEngK8Ar4BdgDOAHNAPEAfsBLwC6AF6AMeAa0A2sdBLAAXABQAFQAMgAcABAAC4AGMANAA1AB4AD6AIYAigBMACeAFUALgAYgAzABvADmAHoAPwAhoBEAESAJYAUYApQBYgC3gGEAYYAyABlADRAHeAPaAfYB-gD_AIoARaAjgCOgEmAJSAUEAq4BYoC0ALSAXMAvIBigDaAG4AOoAfYBDoCLwEqAJ2AUOAo8BTQCrAFsALgAXIAu0BhoDHoGRgZIAyoBlgDMwGcgM-AaIA0gBrA8ACAIocAIABIADIAKAAjgDMgJkATYA2ohAwAAWABQADIALgAYgA1ACGAEwAKYAVQAuABiADMAG8APQAjgBSgCxAGEAMoAd4A-wB_gEUAI4ASkAoIBV4C0ALSAXMAxQBtADqAJUAU0AqwBZQC0QFsALgAXIAu0BkYDOQGeAM-AaIA4AiABATEQACADNAMyAmwBtRKBaAAgABYAFAAMgAcAA-ADEAHgARAAmABVAC4AGIAMwAhoBEAESAI4AUYApQBbgDCAGUAO-AfYB-AEcAKvAWgBaQC5gGKANwAdQA-QCHQEXgKPAWwAu0BkYDLAGcgM8AZ8A0gBrADgCQAMAZoBmQE2FIHQAC4AKAAqABkADgAIAAYwA0ADUAHkAQwBFACYAE8AKQAVQAxABmADmAH4AQ0AiACJAFGAKUAWIAtwBhADIAGUANEAd8A-wD9AItARwBHQCUgFBAKuAVsAuYBeQDaAG4APsAh0BF4CTgE7AKHAVYAtgBcAC5AF2gMNgZGBkgDLAGcgM8AZ8A0iBrAGslQAIAbSgAYAEgAMgA_gEHAJOAmwAA.YAAAAAAAAAAA; consentUUID=1bc4e6ff-da0d-4378-8066-c207a4a5fd8c_20; _sp_v1_data=2:507626:1686356124:0:1:-1:1:0:0:_:-1; _sp_v1_opt=1:login|true:last_id|11:; _sp_v1_csv=; _sp_v1_lt=1:',
        'origin': 'https://www.footballtransfers.com',
        'referer': 'https://www.footballtransfers.com/nl/spelers/nl-eredivisie',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'x-requested-with': 'XMLHttpRequest',
    }
    
    data = {
        'orderBy': 'ft_most_valuable_players.estimated_value',
        'orderByDescending': '1',
        'page': f'{page}',
        'pages': '0',
        'pageItems': '25',
        'continentId': 'all',
        'countryId': 'all',
        'tournamentId': '23',
    }
    
    response = requests.post(
        'https://www.footballtransfers.com/nl/players/actions/overview/overview',
        headers=headers,
        data=data,
    )

    print(f"Page: {page} retrieved!")

    jeZoon = response.json()

    for i in range(25):
        slugs.append(jeZoon["records"][i]['player_slug'])



#print(jeZoon)
#print(jeZoon["records"][0]['player_slug'])

Waiting for 1  seconds.
Page: 1 retrieved!
Waiting for 2  seconds.
Page: 2 retrieved!
Waiting for 3  seconds.
Page: 3 retrieved!
Waiting for 2  seconds.
Page: 4 retrieved!
Waiting for 3  seconds.
Page: 5 retrieved!
Waiting for 1  seconds.
Page: 6 retrieved!
Waiting for 5  seconds.
Page: 7 retrieved!
Waiting for 3  seconds.
Page: 8 retrieved!
Waiting for 2  seconds.
Page: 9 retrieved!
Waiting for 5  seconds.
Page: 10 retrieved!
Waiting for 4  seconds.
Page: 11 retrieved!
Waiting for 8  seconds.
Page: 12 retrieved!
Waiting for 3  seconds.
Page: 13 retrieved!
Waiting for 8  seconds.
Page: 14 retrieved!
Waiting for 7  seconds.
Page: 15 retrieved!
Waiting for 8  seconds.
Page: 16 retrieved!
Waiting for 1  seconds.
Page: 17 retrieved!
Waiting for 5  seconds.
Page: 18 retrieved!


In [5]:
for s in slugs:
    df2.loc[len(df2)] = s
    
df2

,Slug
0,hirving-lozano
1,steven-bergwijn
2,noa-noll-lang
3,sergino-dest
4,kenneth-ina-dorothea-taylor
...,...
445,marcelencio-esajas
446,przemyslaw-tyton
447,sergi-rosanas-moragas
448,boy-waterman


In [6]:
# Save the DataFrame to an Excel file
df2.to_excel('slugs.xlsx', index=False)

print(df2)

                            Slug
0                 hirving-lozano
1                steven-bergwijn
2                  noa-noll-lang
3                   sergino-dest
4    kenneth-ina-dorothea-taylor
..                           ...
445           marcelencio-esajas
446             przemyslaw-tyton
447        sergi-rosanas-moragas
448                 boy-waterman
449            melih-ibrahimoglu

[450 rows x 1 columns]


In [ ]:
for slug in slugs:
    Eredivisie = True
    print(slug)
    x = requests.get(f'https://www.footballtransfers.com/nl/spelers/{slug}/stats')
    interval = randint(1,8)
    print("Waiting for",interval," seconds.") # Wait to not spam website API
    time.sleep(interval)
    player_stats=[]
    for group in range(5):
        seizoen_stats = []
        eredivisie_stats =[]
        soup = BeautifulSoup(x.text, 'html.parser')
        a = soup.find(id=f"career-statistics-group-{group+1}") # takes data from ''Algemeen' from html
        print(f"career-statistics-group-{group+1}")
        b = str(a)
        c = b.split(r"</tr>")
        for count, i in enumerate(c):
            #print(i)
            if "2022/2023" in i:
                seizoen_stats.append(count)
            elif "Eredivisie" in i and seizoen_stats and seizoen_stats[-1] in [count-1, count-2, count-3, count-4, count-5]:
                eredivisie_stats.append(i)
            else:
                pass
        #print(eredivisie_stats)
        if len(eredivisie_stats) > 0:
            q = eredivisie_stats[0].split('</td>')
            #print(q[0])
            q.pop(0)
            #print(q)
            for num,stat in enumerate(q):
                if group==0 and num == 0:       #Get WG and split to WG WI
                    soup = BeautifulSoup(stat, 'html.parser') # Parse the HTML string
                    td_element = soup.find('td') # Find the desired elements
                    value1 = td_element.next.strip()  # WG
                    try:
                        value2 = td_element.find('small').text.strip('()')  # Value if there is no WI to prevent error
                    except AttributeError:
                        value2 = 0
                    player_stats = [value1, value2] # Append the values to a list
                elif group==0 and num > 0:
                    #print('Group 0')
                    stat = re.sub("[^0-9]", "",stat)
                    player_stats.append(stat)
                    #print(q)
                elif group>0 and num > 1:
                    #print('NOT Group 0')
                    stat = re.sub("[^0-9]", "",stat)
                    player_stats.append(stat)
                    #print(stat)
                else:
                    #print("Not group 0")
                    pass
            player_stats.pop(-1)
        else:
            Eredivisie = False
            print("Player not in Eredivisie in selected year")
            player_stats=[slug,'','','','','','','','','','','','','','','','','','','','','','','','']
    ##############################################################################
    to_edit = [7, 9, 13, 16, 22, 24, 26]
    for num, i in enumerate(to_edit):
        #print(num, "e Waarde", columns[i+1])
        if len(player_stats[i]) > 1:
            #print(len(player_stats[i]), "< LENGTH of IF")
            first_part = player_stats[i][:-2]  # Extract all but the last two characters
            second_part = player_stats[i][-2:]  # Extract the last two characters
            player_stats[i] = first_part
            #print(first_part, "< VALUE")
            if first_part:
                player_stats.insert((i + 1), str(round((float(second_part) / 100) * (int(first_part)))))
            else:
                player_stats.insert((i + 1), 0)
        else:
            #print(len(player_stats[i]), "< LENGTH of ELSE")
            player_stats.insert((i + 1), 0)
    player_stats = [float(x) if x != '' and Eredivisie else 0 for x in player_stats] #converts list to integers and replace empty values with 0
    
    player_stats.insert(0,slug)
    print(player_stats)
    print(len(player_stats))

    if len(player_stats) < 33:
        player_stats = [slug, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    else:
        pass
    df.loc[len(df)] = player_stats

In [28]:
# Save the DataFrame to an Excel file
df.to_excel('output.xlsx', index=False)

print(df)

                           Slug    WG   WI      MG    G  AST   GK   RK    SH  \
0    jurrin-david-norman-timber  30.0  1.0  2722.0  3.0  2.0  0.0  0.0  19.0   
1                   xavi-simons   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0   
2               steven-bergwijn   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0   
3                 edson-alvarez  31.0  3.0  2469.0  5.0  1.0  5.0  0.0  36.0   
4                   orkun-kokcu  31.0  0.0  2748.0  7.0  9.0  5.0  0.0  74.0   
..                          ...   ...  ...     ...  ...  ...  ...  ...   ...   
588              maxime-awoudja   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0   
589               metehan-guclu   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0   
590                mika-biereth   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0   
591               pascal-kuiper   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0   
592                pedro-aleman   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0   

      SH%  ...      D     D%     DL   D